In [ ]:
import os
import torch
import matplotlib.pyplot as plt
from transformers import AutoImageProcessor, DetrForSegmentation
from pycocotools.coco import COCO
from PIL import Image
import numpy as np

# Maybe you'll need to get this

# !pip install timm
# !pip install git+https://github.com/cocodataset/panopticapi.git
# !pip install torch torchvision transformers pycocotools pillow numpy matplotlib

# Download and make coco 2017 Data

In [ ]:
!wget http://images.cocodataset.org/zips/val2017.zip
!wget http://images.cocodataset.org/annotations/panoptic_annotations_trainval2017.zip

!unzip val2017.zip
!unzip panoptic_annotations_trainval2017.zip
!unzip ./annotations/panoptic_val2017.zip -d ./annotations/

!rm val2017.zip
!rm panoptic_annotations_trainval2017.zip


In [ ]:
# Maintenant, afin d'utiliser l'API COCO dataset, nous devons convertir son format d'annotation panoptique en un format spécifique qui fonctionnerait pour l'API.
# Le fichier source provient de l'api panoptic, mais il était nécessaire de modifier manuellement un fichier source. D'ou l'importation d'un repo github différent ci dessous.


## Library contains some issues with COCO 2017, let's make some change with "panoptic2detection_coco_format"
##### You could also download it separely !wget https://raw.githubusercontent.com/jessyaz/PANOPTIC_COCO_DETR/main/panoptic2detection_coco_format.py
##### and !wget https://raw.githubusercontent.com/jessyaz/PANOPTIC_COCO_DETR/main/panoptic_coco_categories.json

In [ ]:
!wget https://raw.githubusercontent.com/jessyaz/PANOPTIC_COCO_DETR/main/panoptic2detection_coco_format.py
!wget https://raw.githubusercontent.com/jessyaz/PANOPTIC_COCO_DETR/main/panoptic_coco_categories.json

# Transformation pipeline for coco2017data

In [ ]:
!python3 panoptic2detection_coco_format.py --input_json_file './annotations/panoptic_val2017.json' --output_json_file 'panoptic_annotations.json'

# Let's test it with some iOu metrics

In [ ]:
#Basic coco init
ANNOTATIONS_FILE = 'panoptic_annotations.json'
image_dir = './val2017/'
coco = COCO(ANNOTATIONS_FILE)
image_ids = coco.getImgIds()

CONFIDENCE_THRESHOLD = 0.90
NUM_IMAGES_TO_DISPLAY = 3

# Load pretrained detr-resnet weight
image_processor = AutoImageProcessor.from_pretrained("facebook/detr-resnet-50-panoptic")
model = DetrForSegmentation.from_pretrained("facebook/detr-resnet-50-panoptic")


In [ ]:
def calculate_iou(pred_bbox, gt_bbox):

    xmin_pred, ymin_pred, xmax_pred, ymax_pred = pred_bbox

    xmin_gt, ymin_gt, width_gt, height_gt = gt_bbox

    x_left = max(xmin_pred, xmin_gt)
    y_top = max(ymin_pred, ymin_gt)
    x_right = min(xmax_pred, xmin_gt + width_gt)
    y_bottom = min(ymax_pred, ymin_gt + height_gt)

    intersection_area = max(0, x_right - x_left) * max(0, y_bottom - y_top)

    pred_area = (xmax_pred - xmin_pred) * (ymax_pred - ymin_pred)
    gt_area = width_gt * height_gt
    union_area = pred_area + gt_area - intersection_area

    iou = intersection_area / union_area

    return iou

def is_bbox_match(pred_bbox, gt_bbox, iou_threshold=0.5):

    xmin_pred, ymin_pred, xmax_pred, ymax_pred = pred_bbox
    xmin_gt, ymin_gt, width_gt, height_gt = gt_bbox

    x_left = max(xmin_pred, xmin_gt)
    y_top = max(ymin_pred, ymin_gt)
    x_right = min(xmax_pred, xmin_gt + width_gt)
    y_bottom = min(ymax_pred, ymin_gt + height_gt)

    intersection_area = max(0, x_right - x_left) * max(0, y_bottom - y_top)

    pred_area = (xmax_pred - xmin_pred) * (ymax_pred - ymin_pred)
    gt_area = width_gt * height_gt
    union_area = pred_area + gt_area - intersection_area

    iou = intersection_area / union_area

    match = iou >= iou_threshold

    return match

def calculate_PQ(TP, FP, FN, IoU_values):

    PQ = np.sum(IoU_values) / (np.abs(TP) + 0.5*np.abs(FP) + 0.5*np.abs(FN))

    return PQ



# Run it

In [ ]:

for i in range(NUM_IMAGES_TO_DISPLAY):

    image_data = coco.loadImgs(image_ids[i])[0]
    image_path = image_dir + image_data['file_name']

    annotations_ids = coco.getAnnIds(imgIds=image_data['id'])
    annotations = coco.loadAnns(annotations_ids)

    image = Image.open(image_path).convert("RGB")

    inputs = image_processor(images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)

    result = image_processor.post_process_panoptic_segmentation(outputs, target_sizes=[(image.height, image.width)])

    panoptic_seg = result[0]["segmentation"]
    panoptic_segments_info = result[0]["segments_info"]

    fig, ax = plt.subplots()
    ax.imshow(image)
    ax.axis('off')
    ax.set_title('Image {} avec annotations : {}'.format(i+1, image_data['file_name']) )

    label_id_to_name = {category['id']: category['name'] for category in coco.loadCats(coco.getCatIds())}

    iou_values = []

    TP = 0
    FP = 0
    FN = 0

    for segment_info in panoptic_segments_info:
        detected = False

        if segment_info['score'] > CONFIDENCE_THRESHOLD:
            label_id = segment_info['label_id']

            mask = (panoptic_seg == segment_info['id']).cpu().numpy().astype(np.uint8)

            label_name = ( label_id_to_name.get(label_id, str(label_id)) , label_id )

            ys, xs = np.where(mask == 1)
            y_center = np.mean(ys)
            x_center = np.mean(xs)

            ax.text(x_center, y_center, label_name, fontsize=8, color='black', ha='center')

            ymin, xmin = np.min(ys), np.min(xs)
            ymax, xmax = np.max(ys), np.max(xs)
            pred_bbox = [xmin, ymin, xmax, ymax]
            rect = plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, linewidth=2, edgecolor='r', facecolor='none')

            segment_mask = (result[0]['segmentation'] == segment_info['id'])
            nonzero_indices = torch.nonzero(segment_mask, as_tuple=True)

            xmin = torch.min(nonzero_indices[1]).item()
            xmax = torch.max(nonzero_indices[1]).item()
            ymin = torch.min(nonzero_indices[0]).item()
            ymax = torch.max(nonzero_indices[0]).item()
            pred_bbox = [xmin, ymin, xmax, ymax]

            for annotation in annotations:
                gt_bbox = annotation['bbox']
                if annotation['category_id'] == label_id and is_bbox_match(pred_bbox, gt_bbox):
                    ax.imshow(mask, alpha=0.05, cmap='Blues')
                    ax.add_patch(rect)
                    xmin = annotation['bbox'][0]
                    ymin = annotation['bbox'][1]
                    width = annotation['bbox'][2]
                    height = annotation['bbox'][3]
                    label_name_vrai = annotation['category_id']
                    ax.text((xmin + width//2), (ymin+height//2), label_name_vrai, fontsize=8, color='green', ha='center')
                    rectan = plt.Rectangle((xmin, ymin), width,height, linewidth=3, edgecolor='green', facecolor='none')
                    ax.add_patch(rectan)
                    iou = calculate_iou(pred_bbox, gt_bbox)
                    iou_values.append(iou)
                    detected = True
                    TP += 1

            if not detected:
                FP += 1

    FN = len(annotations) - TP
    print("Vrais positifs (TP):", TP)
    print("Faux positifs (FP):", FP)
    print("Faux négatifs (FN):", FN)
    PQ = calculate_PQ(TP, FP, FN, iou_values)
    print("PQ : ", PQ)

    plt.show()
    print("Valeurs d'IoU :", iou_values)
